# TMM_CfM: Usage Examples

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sf219/TMM_CfM/blob/main/notebooks/02_usage_examples.ipynb)

This notebook demonstrates practical applications and advanced usage of the Transfer Matrix Method for Coding for Machines.

## What You'll Learn

1. Advanced transfer matrix operations
2. Real-world application examples
3. Performance optimization techniques
4. Integration with machine learning workflows

## Setup

First, let's set up our environment with the necessary libraries.

In [ ]:
# Install dependencies
!pip install numpy scipy matplotlib scikit-learn

import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from sklearn.preprocessing import normalize

print("Setup complete!")

## Example 1: Multi-Layer Optical System

One of the classic applications of TMM is in modeling multi-layer optical systems. This example demonstrates how to model light propagation through multiple layers.

In [ ]:
class OpticalLayer:
    """
    Represents a single optical layer with its transfer matrix.
    """
    def __init__(self, refractive_index, thickness, wavelength):
        self.n = refractive_index
        self.d = thickness
        self.wavelength = wavelength
        
    def get_transfer_matrix(self):
        """
        Calculate the transfer matrix for this layer.
        Simplified model for demonstration.
        """
        # Phase factor
        delta = 2 * np.pi * self.n * self.d / self.wavelength
        
        # Transfer matrix
        M = np.array([
            [np.cos(delta), -1j * np.sin(delta) / self.n],
            [-1j * self.n * np.sin(delta), np.cos(delta)]
        ])
        return M

# Create a multi-layer system
wavelength = 550  # nm (green light)

layers = [
    OpticalLayer(1.5, 100, wavelength),  # Layer 1
    OpticalLayer(1.3, 150, wavelength),  # Layer 2
    OpticalLayer(1.7, 120, wavelength),  # Layer 3
]

# Calculate total transfer matrix
M_total = np.eye(2, dtype=complex)
for layer in layers:
    M_total = M_total @ layer.get_transfer_matrix()

print("Total Transfer Matrix for Multi-Layer Optical System:")
print(M_total)
print(f"\nDeterminant: {np.linalg.det(M_total):.4f}")

In [ ]:
# Analyze transmission and reflection
def calculate_transmittance(M, n_incident=1.0, n_substrate=1.5):
    """
    Calculate transmittance from transfer matrix.
    """
    # Simplified calculation
    t = 2 * n_incident / (M[0, 0] * n_substrate + M[0, 1] * n_incident * n_substrate / n_incident + 
                           M[1, 0] + M[1, 1] * n_incident / n_substrate)
    T = np.abs(t)**2 * n_substrate / n_incident
    return T

# Calculate for different wavelengths
wavelengths = np.linspace(400, 700, 100)
transmittances = []

for wl in wavelengths:
    layers_wl = [
        OpticalLayer(1.5, 100, wl),
        OpticalLayer(1.3, 150, wl),
        OpticalLayer(1.7, 120, wl),
    ]
    M = np.eye(2, dtype=complex)
    for layer in layers_wl:
        M = M @ layer.get_transfer_matrix()
    T = calculate_transmittance(M)
    transmittances.append(np.real(T))

# Plot
plt.figure(figsize=(10, 6))
plt.plot(wavelengths, transmittances, linewidth=2)
plt.xlabel('Wavelength (nm)', fontsize=12)
plt.ylabel('Transmittance', fontsize=12)
plt.title('Optical Transmittance vs Wavelength', fontsize=14)
plt.grid(True, alpha=0.3)
plt.axvline(x=550, color='green', linestyle='--', alpha=0.5, label='Design wavelength')
plt.legend()
plt.show()

## Example 2: Coding Chain Transformation

This example shows how TMM can be used to model a coding/encoding chain where data passes through multiple transformation stages.

In [ ]:
class CodingStage:
    """
    Represents a single stage in a coding chain.
    """
    def __init__(self, name, transformation_matrix):
        self.name = name
        self.matrix = transformation_matrix
    
    def encode(self, data):
        """Apply the transformation to input data."""
        return self.matrix @ data

# Define different coding stages
# Stage 1: Scaling and rotation
theta1 = np.pi / 6  # 30 degrees
scale1 = 1.2
M1 = scale1 * np.array([
    [np.cos(theta1), -np.sin(theta1)],
    [np.sin(theta1), np.cos(theta1)]
])

# Stage 2: Shearing transformation
M2 = np.array([
    [1, 0.5],
    [0.3, 1]
])

# Stage 3: Compression and rotation
theta3 = -np.pi / 4  # -45 degrees
scale3 = 0.8
M3 = scale3 * np.array([
    [np.cos(theta3), -np.sin(theta3)],
    [np.sin(theta3), np.cos(theta3)]
])

# Create coding stages
stages = [
    CodingStage("Rotation & Scaling", M1),
    CodingStage("Shearing", M2),
    CodingStage("Compression", M3)
]

# Total transformation matrix
M_total_coding = M3 @ M2 @ M1

print("Coding Chain Stages:")
for i, stage in enumerate(stages, 1):
    print(f"\nStage {i}: {stage.name}")
    print(stage.matrix)

print("\nTotal Transformation Matrix:")
print(M_total_coding)

In [ ]:
# Visualize the transformation of a set of data points
# Generate test data (a simple square)
square = np.array([
    [0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0]
])

# Apply transformations step by step
transformed_data = [square]
current_data = square.copy()

for stage in stages:
    current_data = stage.encode(current_data)
    transformed_data.append(current_data.copy())

# Plot the transformation progression
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

titles = ['Original Data', 'After Stage 1', 'After Stage 2', 'After Stage 3']

for idx, (ax, data, title) in enumerate(zip(axes, transformed_data, titles)):
    ax.plot(data[0], data[1], 'b-o', linewidth=2, markersize=8)
    ax.fill(data[0], data[1], alpha=0.3)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    ax.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax.axvline(x=0, color='k', linestyle='-', alpha=0.3)
    
plt.tight_layout()
plt.show()

## Example 3: Machine Learning Integration

This example demonstrates how TMM can be integrated with machine learning for automated optimization.

In [ ]:
from scipy.optimize import minimize

def create_parameterized_matrix(params):
    """
    Create a transfer matrix from parameters.
    params: [theta, scale_x, scale_y, shear]
    """
    theta, scale_x, scale_y, shear = params
    
    # Rotation matrix
    R = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])
    
    # Scaling matrix
    S = np.array([
        [scale_x, 0],
        [0, scale_y]
    ])
    
    # Shear matrix
    H = np.array([
        [1, shear],
        [0, 1]
    ])
    
    return H @ S @ R

# Define optimization objective: minimize difference from target transformation
target_output = np.array([[2.0], [1.5]])
input_data = np.array([[1.0], [1.0]])

def objective(params):
    """Objective function: minimize error to target output."""
    M = create_parameterized_matrix(params)
    output = M @ input_data
    error = np.sum((output - target_output)**2)
    return error

# Initial guess
initial_params = [0.0, 1.0, 1.0, 0.0]  # [theta, scale_x, scale_y, shear]

# Optimize
result = minimize(objective, initial_params, method='BFGS')

print("Optimization Results:")
print(f"Success: {result.success}")
print(f"Optimal parameters:")
print(f"  Rotation angle: {result.x[0]:.4f} rad ({np.degrees(result.x[0]):.2f}°)")
print(f"  Scale X: {result.x[1]:.4f}")
print(f"  Scale Y: {result.x[2]:.4f}")
print(f"  Shear: {result.x[3]:.4f}")
print(f"\nFinal error: {result.fun:.6f}")

# Verify the result
optimal_matrix = create_parameterized_matrix(result.x)
optimal_output = optimal_matrix @ input_data

print(f"\nTarget output: {target_output.flatten()}")
print(f"Achieved output: {optimal_output.flatten()}")
print(f"\nOptimized Transfer Matrix:")
print(optimal_matrix)

## Example 4: Performance Analysis

Let's analyze the computational performance of TMM operations with different system sizes.

In [ ]:
import time

def benchmark_tmm(num_layers, matrix_size=2):
    """
    Benchmark TMM computation for a given number of layers.
    """
    # Generate random transfer matrices
    matrices = [np.random.randn(matrix_size, matrix_size) for _ in range(num_layers)]
    
    # Time the multiplication
    start_time = time.time()
    
    M_total = np.eye(matrix_size)
    for M in matrices:
        M_total = M_total @ M
    
    elapsed_time = time.time() - start_time
    return elapsed_time

# Test different numbers of layers
layer_counts = [10, 50, 100, 500, 1000, 5000]
computation_times = []

print("Performance Benchmarking:")
print("=" * 40)

for num_layers in layer_counts:
    # Run multiple times and average
    times = [benchmark_tmm(num_layers) for _ in range(5)]
    avg_time = np.mean(times)
    computation_times.append(avg_time)
    print(f"{num_layers:5d} layers: {avg_time*1000:8.3f} ms")

# Plot performance
plt.figure(figsize=(10, 6))
plt.plot(layer_counts, np.array(computation_times)*1000, 'o-', linewidth=2, markersize=8)
plt.xlabel('Number of Layers', fontsize=12)
plt.ylabel('Computation Time (ms)', fontsize=12)
plt.title('TMM Performance vs Number of Layers', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.yscale('log')
plt.show()

print(f"\nScaling: approximately O(n) for {matrix_size}x{matrix_size} matrices")

## Best Practices

When working with TMM_CfM, consider these best practices:

### 1. Numerical Stability
- Monitor matrix condition numbers
- Use appropriate numerical precision
- Consider regularization for ill-conditioned systems

### 2. Performance Optimization
- Pre-compute matrices when possible
- Use vectorization for batch operations
- Consider sparse matrix representations for large systems

### 3. Validation
- Always verify physical constraints (e.g., energy conservation)
- Test edge cases
- Compare with analytical solutions when available

In [ ]:
# Example: Checking numerical stability
def check_stability(matrix, threshold=1e10):
    """
    Check if a matrix is numerically stable.
    """
    condition_number = np.linalg.cond(matrix)
    is_stable = condition_number < threshold
    
    print(f"Condition number: {condition_number:.2e}")
    print(f"Numerically stable: {is_stable}")
    
    if not is_stable:
        print("⚠️  Warning: Matrix may be ill-conditioned!")
    else:
        print("✓ Matrix is well-conditioned")
    
    return is_stable, condition_number

# Test our optical system matrix
print("Stability check for optical system:")
check_stability(M_total)

print("\nStability check for coding chain:")
check_stability(M_total_coding)

## Conclusion

In this notebook, you've learned:

- ✓ How to apply TMM to optical systems
- ✓ Using TMM for coding chain transformations
- ✓ Integrating TMM with machine learning optimization
- ✓ Performance analysis and benchmarking
- ✓ Best practices for numerical stability

## Next Steps

- Explore your own applications of TMM
- Contribute improvements to the repository
- Check out the [project documentation](https://sf219.github.io/TMM_CfM/)
- Share your findings with the community

## Resources

- [Introduction Notebook](01_introduction.ipynb)
- [GitHub Repository](https://github.com/sf219/TMM_CfM)
- [Documentation](https://sf219.github.io/TMM_CfM/)